### NLP
Sentiment Analysis


* 00) Definition
* 01) Overall Sentiment score
* 02) VADER Amazon Reviews
  * 2.1) Load Data
  * 2.2) Clean the data (optional)
  * 2.3) Score segun Vader en la primera reseña
  * 2.4) Generación columna Compound
  * 2.5) Report on Accuracy


## 0) Paquetes Necesarios





# Sentiment Analysis
Now that we've seen word vectors we can start to investigate sentiment analysis. The goal is to find commonalities between documents, with the understanding that similarly *combined* vectors should correspond to similar sentiments.

While the scope of sentiment analysis is very broad, we will focus our work in two ways.

### 1. Polarity classification
We won't try to determine if a sentence is objective or subjective, fact or opinion. Rather, we care only if the text expresses a *positive*, *negative* or *neutral* opinion.
### 2. Document level scope
We'll also try to aggregate all of the sentences in a document or paragraph, to arrive at an overall opinion.
### 3. Coarse analysis
We won't try to perform a fine-grained analysis that would determine the degree of positivity/negativity. That is, we're not trying to guess how many stars a reviewer awarded, just whether the review was positive or negative.

## Broad Steps:
* First, consider the text being analyzed. A model trained on paragraph-long movie reviews might not be effective on tweets. Make sure to use an appropriate model for the task at hand.
* Next, decide the type of analysis to perform. In the previous section on text classification we used a bag-of-words technique that considered only single tokens, or *unigrams*. Some rudimentary sentiment analysis models go one step further, and consider two-word combinations, or *bigrams*. In this section, we'd like to work with complete sentences, and for this we're going to import a trained NLTK lexicon called *VADER*.

## NLTK's VADER module
VADER is an NLTK module that provides sentiment scores based on words used ("completely" boosts a score, while "slightly" reduces it), on capitalization & punctuation ("GREAT!!!" is stronger than "great."), and negations (words like "isn't" and "doesn't" affect the outcome).
<br>To view the source code visit https://www.nltk.org/_modules/nltk/sentiment/vader.html

**Download the VADER lexicon.** You only need to do this once.

## 01) Paquetes Necesarios


In [ ]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
sid = SentimentIntensityAnalyzer()

## 01) Overall Sentiment score

VADER's `SentimentIntensityAnalyzer()` takes in a string and returns a dictionary of scores in each of four categories:
* negative
* neutral
* positive
* compound *(computed by normalizing the scores above)*

In [ ]:
a = 'This was a good movie.'
sid.polarity_scores(a)

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

In [ ]:
a = 'This was the best, most awesome movie EVER MADE!!!'
sid.polarity_scores(a)

{'neg': 0.0, 'neu': 0.425, 'pos': 0.575, 'compound': 0.8877}

In [ ]:
a = 'This was the worst film to ever disgrace the screen.'
sid.polarity_scores(a)

{'neg': 0.477, 'neu': 0.523, 'pos': 0.0, 'compound': -0.8074}

Como podemos ver, despues de introducir una frase, vader tiene la capacidad de identificar el score positivo, neutro y negativo. Dando lugar una punctuacion final la cual denomina compoung.

Este muestra valores positivos si la polaridad del mismo en conjunto es positivo y negativo en caso contrario.

##  2) Use VADER to analyze Amazon Reviews

En este ejercicio aplicaremos `SentimentIntensityAnalyzer` para analizar un dataset de amazon compuesto por 10000 reseñas-

Estas reseñas tendrá asociado una etiqueta "pos" o "neg" la cual utilizaremos únicamente para calcular el porcetaje de acierto que muestra VADER.

Es decir las etiquetas NO intervienen en el proceso de clasificación.

### 2.1) Load Data

In [ ]:
"""
In this chunk we define the relative path for our dataset.
"""

from google.colab import drive

mount='/content/gdrive'
print("Colab: mounting Google drive on ", mount)

drive.mount(mount)

# Switch to the directory on the Google Drive that you want to use
import os
drive_root = mount + "/My Drive/Colab Notebooks/00_CEU/NLP/Chap_5_Sentiment_Analysis/dataset/"

# Create drive_root if it doesn't exist
create_drive_root = True
if create_drive_root:
  print("\nColab: making sure ", drive_root, " exists.")
  os.makedirs(drive_root, exist_ok=True)

# Change to the directory
print("\nColab: Changing directory to ", drive_root)
%cd $drive_root

Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/Colab Notebooks/00_CEU/NLP/Chap_5_Sentiment_Analysis/dataset/  exists.

Colab: Changing directory to  /content/gdrive/My Drive/Colab Notebooks/00_CEU/NLP/Chap_5_Sentiment_Analysis/dataset/
/content/gdrive/My Drive/Colab Notebooks/00_CEU/NLP/Chap_5_Sentiment_Analysis/dataset


In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('amazonreviews.tsv', sep='\t')
df.head()

label                                             review
0   pos  Stuning even for the non-gamer: This sound tra...
1   pos  The best soundtrack ever to anything.: I'm rea...
2   pos  Amazing!: This soundtrack is my favorite music...
3   pos  Excellent Soundtrack: I truly like this soundt...
4   pos  Remember, Pull Your Jaw Off The Floor After He...

In [ ]:
df['label'].value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

### 2.2) Clean the data (optional):

Este paso es opcional, pero es recomendable eliminar todos los registros en blanco si los hubiera.

In [ ]:
# REMOVE NaN VALUES AND EMPTY STRINGS:
df.dropna(inplace=True)

blanks = []  # start with an empty list

for i,lb,rv in df.itertuples():  # iterate over the DataFrame
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

df.drop(blanks, inplace=True)

In [ ]:
df['label'].value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

En este caso no existen registros en blanco

### 2.3) Score segun Vader en la primera reseña

In [ ]:
df.loc[0]

label                                                   pos
review    Stuning even for the non-gamer: This sound tra...
Name: 0, dtype: object

In [ ]:
sid.polarity_scores(df.loc[0]['review'])

{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'compound': 0.9454}

In [ ]:
df.loc[0]['label']

'pos'

Según Vader la reseña tiene un sentimiento Positivo

### 2.4) Adding Scores and Labels to the DataFrame

Se añade una nueva columna indicando el socere segun vader en cada reseña. El motivo de añadir esta última columna es para poder validar el score de cada reseña con respecto a la realidad




In [ ]:
df['scores'] = df['review'].apply(lambda review: sid.polarity_scores(review))
df.head()

label                                             review  \
0   pos  Stuning even for the non-gamer: This sound tra...   
1   pos  The best soundtrack ever to anything.: I'm rea...   
2   pos  Amazing!: This soundtrack is my favorite music...   
3   pos  Excellent Soundtrack: I truly like this soundt...   
4   pos  Remember, Pull Your Jaw Off The Floor After He...   

                                              scores  
0  {'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...  
1  {'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...  
2  {'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...  
3  {'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...  
4  {'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...

In [ ]:
# Nos creamos una nueva columna con el valor de compound
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
df.head()

label                                             review  \
0   pos  Stuning even for the non-gamer: This sound tra...   
1   pos  The best soundtrack ever to anything.: I'm rea...   
2   pos  Amazing!: This soundtrack is my favorite music...   
3   pos  Excellent Soundtrack: I truly like this soundt...   
4   pos  Remember, Pull Your Jaw Off The Floor After He...   

                                              scores  compound  
0  {'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...    0.9454  
1  {'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...    0.8957  
2  {'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...    0.9858  
3  {'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...    0.9814  
4  {'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...    0.9781

In [ ]:
# Se genera la columna com_score para accuracy
df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

df.head()

label                                             review  \
0   pos  Stuning even for the non-gamer: This sound tra...   
1   pos  The best soundtrack ever to anything.: I'm rea...   
2   pos  Amazing!: This soundtrack is my favorite music...   
3   pos  Excellent Soundtrack: I truly like this soundt...   
4   pos  Remember, Pull Your Jaw Off The Floor After He...   

                                              scores  compound comp_score  
0  {'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...    0.9454        pos  
1  {'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...    0.8957        pos  
2  {'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...    0.9858        pos  
3  {'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...    0.9814        pos  
4  {'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...    0.9781        pos

#### 2.5) Report on Accuracy
Finally, we'll use scikit-learn to determine how close VADER came to our original 10,000 labels.

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
accuracy_score(df['label'],df['comp_score'])

0.7097

In [ ]:
print(classification_report(df['label'],df['comp_score']))

              precision    recall  f1-score   support

         neg       0.86      0.52      0.64      5097
         pos       0.64      0.91      0.75      4903

    accuracy                           0.71     10000
   macro avg       0.75      0.71      0.70     10000
weighted avg       0.75      0.71      0.70     10000



In [ ]:
print(confusion_matrix(df['label'],df['comp_score']))

[[2629 2468]
 [ 435 4468]]


In [ ]:
print('En este caso podemos que que tenemos un total de ', (2629+4468)/(2629+4468+435+2468),'% de correctamente clasificadas')

En este caso podemos que que tenemos un total de  0.7097 %


#--